<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from IPython.display import display

"""
Анализ устойчивости распределения P(next_token) к изменениям prompt‑ов.
---------------------------------------------------------------
* **Строки таблиц** = сами тексты prompt‑ов (англ. или рус.)
* **Столбцы** = названия модификаций (можно задавать по‑русски).
* Число промптов и модификаций произвольное.

Запуск в Jupyter:
    %run analyse_positional_stability_GPT_multi_prompts.py
"""

# ---------- 1. Загрузка модели ----------
MODEL_NAME = "gpt2"                                # ↺ при необходимости замените
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"

_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
_model     = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE).eval()

# ---------- 2. Вспомогательные функции ----------

def _next_token_probs(prompt: str):
    """P(next_token | prompt) как 1‑D тензор softmax‑вероятностей длиной |V|."""
    ids = _tokenizer.encode(prompt, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = _model(ids).logits  # [1, seq_len, |V|]
    return torch.softmax(logits[0, -1], dim=-1).cpu()


def _kl_div(p, q):
    return F.kl_div(p.log(), q, reduction="batchmean").item()


def _cos_sim(p, q):
    return F.cosine_similarity(p, q, dim=0).item()

# ---------- 3. Исходные промпты ----------
#   Просто список строк — они же будут строками итоговых таблиц
PROMPTS = [
    "Why the stock market is expected to",
    "The future of artificial intelligence depends on",
    # добавьте свои prompt‑ы ниже (можно русские)
]

# ---------- 4. Модификации промптов ----------
#   Формат:  "Название_модификации" : lambda prompt: <новый_prompt>
MODS = {
    "префикс ======":   lambda p: "=" * 10 + p,
    "префикс вопрос":  lambda p: "I have a question. " + p,
    "суффикс 10 лет?": lambda p: p + " in the next decade?",
    # добавьте свои модификации
}

# ---------- 5. Расчёт метрик ----------
records = []
for prompt in PROMPTS:
    base_probs = _next_token_probs(prompt)
    for mod_name, mod_fn in MODS.items():
        mod_prompt  = mod_fn(prompt)
        mod_probs   = _next_token_probs(mod_prompt)
        records.append({
            "prompt_text": prompt,
            "mod_name":    mod_name,
            "cosine":      _cos_sim(base_probs, mod_probs),
            "kl":          _kl_div(base_probs, mod_probs),
        })

_df = pd.DataFrame(records)

# ---------- 6. Пивот‑таблицы ----------
cos_table = _df.pivot(index="prompt_text", columns="mod_name", values="cosine")
kl_table  = _df.pivot(index="prompt_text", columns="mod_name", values="kl")

pd.set_option("display.precision", 4)

# ---------- 7. Вывод ----------
print("\n▶ Cosine similarity (ближе к 1 — больше сходства)")
display(cos_table)

print("\n▶ Kullback–Leibler divergence (меньше — ближе)\n")
display(kl_table)

# ---------- 8. (Опц.) Сохранить результаты ----------
# cos_table.to_csv("cosine_similarity.csv")
# kl_table.to_csv("kl_divergence.csv")


C:\Users\cab\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\cab\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (



▶ Cosine similarity (ближе к 1 — больше сходства)


mod_name,префикс ======,префикс вопрос,суффикс 10 лет?
prompt_text,,,
The future of artificial intelligence depends on,0.9745,0.9395,0.0024
Why the stock market is expected to,0.9017,0.9308,0.0005



▶ Kullback–Leibler divergence (меньше — ближе)



mod_name,префикс ======,префикс вопрос,суффикс 10 лет?
prompt_text,,,
The future of artificial intelligence depends on,9.6809e-07,2.2507e-06,0.0001
Why the stock market is expected to,1.9205e-06,1.7422e-06,0.0002
